In [ ]:
import numpy as np

def lu_decomposition(A):
    n = A.shape[0]
    L = np.eye(n, dtype=float)
    U = np.zeros((n, n), dtype=float)

    for i in range(n):
        for k in range(i, n):
            U[i, k] = A[i, k] - sum(L[i, j] * U[j, k] for j in range(i))
        # Computa coluna i de L
        if abs(U[i, i]) < 1e-12:
            raise ZeroDivisionError(f"Pivô quase zero em U[{i},{i}]: {U[i,i]}")
        for k in range(i+1, n):
            L[k, i] = (A[k, i] - sum(L[k, j] * U[j, i] for j in range(i))) / U[i, i]

    return L, U

# Substituição direta para Ly = b
def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b, dtype=float)
    for i in range(n):
        soma = sum(L[i, j] * y[j] for j in range(i))
        y[i] = (b[i] - soma) / L[i, i]
    return y

# Substituição reversa para Ux = y
def backward_substitution(U, y):
    n = U.shape[0]
    x = np.zeros_like(y, dtype=float)
    for i in range(n-1, -1, -1):
        soma = sum(U[i, j] * x[j] for j in range(i+1, n))
        x[i] = (y[i] - soma) / U[i, i]
    return x

def solve_lu(A, b):
    """
    Resolve Ax = b usando decomposição LU e substituições.
    """
    L, U = lu_decomposition(A)
    print("Matriz L:")
    print(L)
    print('------------------------------')
    print("Matriz U:")
    print(U)
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)
    return x

if __name__ == '__main__':
    A = np.array([[4, 3, 2],
                [8, 8, 5],
                [4, 6, 9]], dtype=float)
    b = np.array([1, 2, 5], dtype=float)

    print("Matriz A:")
    print(A)
    print("Vetor b:")
    print(b)
    print("------------------------------")

    try:
        x = solve_lu(A, b)
        print('')
        print("Solução x:")
        print(x)
    except Exception as err:
        print("Erro ao resolver sistema:", err)




Matriz A:
[[4. 3. 2.]
 [8. 8. 5.]
 [4. 6. 9.]]
Vetor b:
[1. 2. 5.]
------------------------------
Matriz L:
[[1.  0.  0. ]
 [2.  1.  0. ]
 [1.  1.5 1. ]]
------------------------------
Matriz U:
[[4.  3.  2. ]
 [0.  2.  1. ]
 [0.  0.  5.5]]

Solução x:
[ 0.15909091 -0.36363636  0.72727273]


#### a) Em que situações a decomposição LU falha ou se torna numericamente instável?

- Quando a matriz **A** é singular (determinante zero) ou quase singular (determinante muito pequeno), o método falha.
- Quando algum pivô **U[i][i]** é zero ou muito próximo de zero, ocorre divisão por zero ou instabilidade numérica.
- Matrizes mal-condicionadas (número de condição elevado) levam a resultados imprecisos.

---

#### b) Como o pivotamento parcial ajuda?

- O pivotamento parcial troca linhas para que o maior valor (em módulo) na coluna corrente fique na diagonal.
- Isso evita divisões por números pequenos e melhora a estabilidade numérica do algoritmo.
- O método passa a usar uma matriz de permutação **P** tal que **PA = LU**, em vez de **A = LU**.

In [ ]:
# Questão 4


import numpy as np

def lu_decomposition(A):
    A = np.array(A, dtype=float)
    n = A.shape[0]
    if A.ndim != 2 or A.shape[1] != n:
        raise ValueError("A deve ser matriz quadrada.")
    L = np.eye(n, dtype=float)
    U = np.zeros((n, n), dtype=float)
    for i in range(n):
        for j in range(i, n):
            U[i, j] = A[i, j] - sum(L[i, k] * U[k, j] for k in range(i))
        if abs(U[i, i]) < 1e-12:
            raise np.linalg.LinAlgError(f"Pivô quase zero U[{i},{i}].")
        for j in range(i+1, n):
            L[j, i] = (A[j, i] - sum(L[j, k] * U[k, i] for k in range(i))) / U[i, i]
    return L, U

def forward_substitution(L, b):
    L = np.array(L, dtype=float)
    b = np.array(b, dtype=float)
    n = L.shape[0]
    y = np.zeros(n, dtype=float)
    for i in range(n):
        if abs(L[i, i]) < 1e-12:
            raise np.linalg.LinAlgError(f"Pivô quase zero L[{i},{i}].")
        y[i] = (b[i] - sum(L[i, k] * y[k] for k in range(i))) / L[i, i]
    return y

def backward_substitution(U, y):
    U = np.array(U, dtype=float)
    y = np.array(y, dtype=float)
    n = U.shape[0]
    x = np.zeros(n, dtype=float)
    for i in range(n-1, -1, -1):
        if abs(U[i, i]) < 1e-12:
            raise np.linalg.LinAlgError(f"Pivô quase zero U[{i},{i}].")
        x[i] = (y[i] - sum(U[i, k] * x[k] for k in range(i+1, n))) / U[i, i]
    return x


def solve_linear_system(A, b):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    if A.shape[0] != A.shape[1]:
        raise ValueError("A deve ser quadrada.")
    if b.size != A.shape[0]:
        raise ValueError("Dimensões incompatíveis.")
    L, U = lu_decomposition(A)
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)
    return x

def inverse_matrix(A):
    A = np.array(A, dtype=float)
    n = A.shape[0]
    if A.ndim != 2 or A.shape[1] != n:
        raise ValueError("A deve ser matriz quadrada.")
    invA = np.zeros((n, n), dtype=float)
    I = np.eye(n, dtype=float)
    for i in range(n):
        invA[:, i] = solve_linear_system(A, I[:, i])
    return invA


def norm1(A):
    A = np.array(A, dtype=float)
    return np.max(np.sum(np.abs(A), axis=0))


def condition_number(A):
    A = np.array(A, dtype=float)
    return norm1(A) * norm1(inverse_matrix(A))


def concentrations(A, b):
    return solve_linear_system(A, b)


def reduction_needed_for_target_c4(A, b, target):
    invA = inverse_matrix(A)
    coeffs = invA[3, :]
    rest = sum(coeffs[j] * b[j] for j in range(1, 4))
    b1_new = (target - rest) / coeffs[0]
    return b[0] - b1_new


def conditioning_info(A):
    cond = condition_number(A)
    digits = int(np.floor(np.log10(cond))) if cond > 0 else None
    return cond, digits


def main():
    A = [
        [13.422,  0,      0,      0],
        [-13.422, 12.252, 0,      0],
        [0,      -12.252, 12.377, 0],
        [0,       0,     -12.377, 11.797]
    ]
    b = [750.5, 300, 102, 30]

    try:
        c = concentrations(A, b)
        print("c:", np.round(c, 6))
        red = reduction_needed_for_target_c4(A, b, 75.0)
        print("Redução c4 para 75 deve ser:", round(red, 6))
        cond, digs = conditioning_info(A)
        print("cond1:", round(cond, 6), "dígitos suspeitos:", digs)
    except Exception as e:
        print("Erro:", e)

if __name__ == "__main__":
    main()


c: [ 55.915661  85.741103  93.116264 100.237348]
Redução c4 para 75 deve ser: 297.725
cond1: 8.635345 dígitos suspeitos: 0


In [78]:
import numpy as np

# Decomposição LU sem pivotamento
def lu_decomposition(A):
    n = A.shape[0]
    L = np.eye(n, dtype=float)
    U = np.zeros((n, n), dtype=float)

    for i in range(n):
        for k in range(i, n):
            U[i, k] = A[i, k] - sum(L[i, j] * U[j, k] for j in range(i))
        # Computa coluna i de L
        if abs(U[i, i]) < 1e-12:
            raise ZeroDivisionError(f"Pivô quase zero em U[{i},{i}]: {U[i,i]}")
        for k in range(i+1, n):
            L[k, i] = (A[k, i] - sum(L[k, j] * U[j, i] for j in range(i))) / U[i, i]

    return L, U

# Substituição direta para Ly = b
def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b, dtype=float)
    for i in range(n):
        soma = sum(L[i, j] * y[j] for j in range(i))
        y[i] = (b[i] - soma) / L[i, i]
    return y

# Substituição reversa para Ux = y
def backward_substitution(U, y):
    n = U.shape[0]
    x = np.zeros_like(y, dtype=float)
    for i in range(n-1, -1, -1):
        soma = sum(U[i, j] * x[j] for j in range(i+1, n))
        x[i] = (y[i] - soma) / U[i, i]
    return x

def solve_lu(A, b):
    """
    Resolve Ax = b usando decomposição LU e substituições.
    """
    L, U = lu_decomposition(A)
    print("Matriz L:")
    print(L)
    print('------------------------------')
    print("Matriz U:")
    print(U)
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)
    return x

if __name__ == '__main__':
    A = np.array([[6, 2, 1],
                [3, 5, 1],
                [2, 1, 4]], dtype=float)
    b = np.array([9, 10, 12], dtype=float)

    print("Matriz A:")
    print(A)
    print("Vetor b:")
    print(b)
    print("------------------------------")

    try:
        x = solve_lu(A, b)
        print('')
        print("Solução x:")
        print(x)
    except Exception as err:
        print("Erro ao resolver sistema:", err)




Matriz A:
[[6. 2. 1.]
 [3. 5. 1.]
 [2. 1. 4.]]
Vetor b:
[ 9. 10. 12.]
------------------------------
Matriz L:
[[1.         0.         0.        ]
 [0.5        1.         0.        ]
 [0.33333333 0.08333333 1.        ]]
------------------------------
Matriz U:
[[6.    2.    1.   ]
 [0.    4.    0.5  ]
 [0.    0.    3.625]]

Solução x:
[0.74712644 1.08045977 2.35632184]


In [77]:
# Questão 6


import numpy as np

def lu_decomposition(A):
    n = A.shape[0]
    L = np.eye(n, dtype=float)
    U = np.zeros((n, n), dtype=float)
    for i in range(n):
        for k in range(i, n):
            U[i, k] = A[i, k] - sum(L[i, j] * U[j, k] for j in range(i))
        if abs(U[i, i]) < 1e-12:
            raise ZeroDivisionError(f"Pivô quase zero em U[{i},{i}]: {U[i,i]}")
        for k in range(i+1, n):
            L[k, i] = (A[k, i] - sum(L[k, j] * U[j, i] for j in range(i))) / U[i, i]
    return L, U

def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros(n, dtype=float)
    for i in range(n):
        y[i] = (b[i] - sum(L[i, j] * y[j] for j in range(i))) / L[i, i]
    return y

def backward_substitution(U, y):
    n = U.shape[0]
    x = np.zeros(n, dtype=float)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - sum(U[i, j] * x[j] for j in range(i+1, n))) / U[i, i]
    return x

def solve_lu(A, b):
    L, U = lu_decomposition(A)
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)
    return x

# Parâmetros do sistema
k1 = k4 = 10.0  # N/m
k2 = k3 = 30.0  # N/m
m1 = m2 = m3 = 2.0  # kg

# Matrizes M e K
M = np.diag([m1, m2, m3])
K = np.array([
    [k1 + k2,    -k2,       0    ],
    [   -k2,  k2 + k3,    -k3    ],
    [    0,      -k3,   k3 + k4  ]
], dtype=float)

# Deslocamentos no instante dado
x = np.array([0.05, 0.04, 0.03], dtype=float)

# Monta b = -K x
b = -K.dot(x)

# Resolve M a = b
a = solve_lu(M, b)


print("\nAcelerações a = [a1, a2, a3]:")
print(a)



Acelerações a = [a1, a2, a3]:
[-4.00000000e-01  8.32667268e-17 -0.00000000e+00]
